<a href="https://colab.research.google.com/github/fazal735/DL_A2/blob/main/DL_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import torchvision
from torch.utils.data import DataLoader,SubsetRandomSampler
from sklearn.model_selection import train_test_split


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
train_data_dir = '/content/drive/MyDrive/inaturalist_12K/train'
test_data_dir = '/content/drive/MyDrive/inaturalist_12K/val'

In [33]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [22]:

# train data loading
def train_data(train_data_dir,data_augmentation):
  size=transforms.Resize((224,224))
  to_tensor=transforms.ToTensor()
  #check again-autogenerated
  normalize=transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
  crop=transforms.RandomResizedCrop(224)
  flip=transforms.RandomHorizontalFlip()
  #try changing the values
  color=transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
  rotation=transforms.RandomRotation(30)


  #augmentation of image
  if data_augmentation == 'Yes':
        transform_img = transforms.Compose([crop,flip,color, rotation, to_tensor,normalize]) # Data transformations

  else:
      transform_img = transforms.Compose([size,to_tensor, normalize ])



  #data fetchiing
  training_data=torchvision.datasets.ImageFolder(train_data_dir, transform=transform_img)


  train_index, val_index = train_test_split(list(range(len(training_data))), test_size=0.2, random_state=42)
  train_sampler = SubsetRandomSampler(train_index)
  val_sampler = SubsetRandomSampler(val_index)

  train_data=DataLoader(training_data,batch_size=32,sampler=train_sampler)
  validation_data = DataLoader(training_data, batch_size=32, sampler=val_sampler)


  return train_data,validation_data

#test data loading
def test_data(test_data_dir,data_augmentation):
  size=transforms.Resize((224,224))
  to_tensor=transforms.ToTensor()
  #check again-autogenerated
  normalize=transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
  crop=transforms.RandomResizedCrop(224)
  flip=transforms.RandomHorizontalFlip()
  #try changing the values
  color=transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
  rotation=transforms.RandomRotation(30)

  #augmentation of image
  data_augmentation=transforms.Compose([size,to_tensor,normalize,crop,flip,color,rotation])

  #data fetching
  test_data=torchvision.datasets.ImageFolder(test_data_dir)
  test_data=DataLoader(test_data,batch_size=32)
  print(test_data.type)

  return test_data

model definition

In [23]:
evice, but foun# model definetion
class model(nn.Module):
  def __init__(self,num_filters=[32,64,128,256,512],filter_size=[3,3,5,5,7],activation=nn.ReLU(),
               stride=1, padding=1, pool_size=(2,2),fc_size=512,nom_o_classes=10,
               dropout=0,in_channels=3,batch_norm='YES'):
    super(model,self).__init__()
    self.num_filters=num_filters
    self.filter_size=filter_size
    self.activation=activation
    self.stride=stride
    self.padding=padding
    self.pool_size=pool_size
    self.fc_size=fc_size
    self.nom_o_classes=nom_o_classes
    self.dropout=dropout
    self.channels=in_channels


    def image_size(img_w,filter_size,padding,stride):
      return ((img_w-filter_size+2*padding)/stride+1)*0.5


    #layers of convolution
    #layer1
    self.conv_layer1=nn.Conv2d(self.channels,self.num_filters[0], stride=self.stride, padding=self.padding,
                               kernel_size=self.filter_size[0])
    self.batch_norm1=nn.BatchNorm2d(self.num_filters[0])
    self.dropout1=nn.Dropout2d(self.dropout)

    img_size1=image_size(224,self.filter_size[0],self.padding,self.stride)

    #layer2
    self.conv_layer2=nn.Conv2d(self.num_filters[0],self.num_filters[1], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[1])
    self.batch_norm2=nn.BatchNorm2d(self.num_filters[1])
    self.dropout2=nn.Dropout2d(self.dropout)

    img_size2=image_size(img_size1,self.filter_size[1],self.padding,self.stride)

    #layer3
    self.conv_layer3=nn.Conv2d(self.num_filters[1],self.num_filters[2], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[2])
    self.batch_norm3=nn.BatchNorm2d(self.num_filters[2])
    self.dropout3=nn.Dropout2d(self.dropout)

    img_size3=image_size(img_size2,self.filter_size[2],self.padding,self.stride)

    #layer4
    self.conv_layer4=nn.Conv2d(self.num_filters[2],self.num_filters[3], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[3])
    self.batch_norm4=nn.BatchNorm2d(self.num_filters[3])
    self.dropout4=nn.Dropout2d(self.dropout)

    img_size4=image_size(img_size3,self.filter_size[3],self.padding,self.stride)

    #layer5
    self.conv_layer5=nn.Conv2d(self.num_filters[3],self.num_filters[4], stride=self.stride, padding=self.padding,
                              kernel_size=self.filter_size[4])
    self.batch_norm5=nn.BatchNorm2d(self.num_filters[4])
    self.dropout5=nn.Dropout2d(self.dropout)

    img_size5=int(image_size(img_size4,self.filter_size[4],self.padding,self.stride))


    self.pool=nn.MaxPool2d(self.pool_size,stride=2)

    self.dropout_layer = nn.Dropout1d(self.dropout)

    # Define fully connected layer
    self.fc = nn.Linear(self.num_filters[4] * (img_size5 ** 2), self.fc_size)
    self.fc_bn = nn.BatchNorm1d(self.fc_size)  # Batch normalization for fully connected layer

    # Output layer
    self.output_layer = nn.Linear(self.fc_size, self.nom_o_classes)

    # forward
  def forward(self,x):
    #layer1
    x=self.conv_layer1(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout1(x)
    x=self.batch_norm1(x)

      #layer2
    x=self.conv_layer2(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout2(x)
    x=self.batch_norm2(x)

    #layer3
    x=self.conv_layer3(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout3(x)
    x=self.batch_norm3(x)

    #layer4
    x=self.conv_layer4(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout4(x)
    x=self.batch_norm4(x)

    #layer5
    x=self.conv_layer5(x)
    x=self.activation(x)
    x=self.pool(x)
    x=self.dropout5(x)
    x=self.batch_norm5(x)

    x = torch.flatten(x, 1)
    x = self.fc(x)
    x = self.fc_bn(x)
    x = self.activation(x)
    x = self.dropout_layer(x)
    x = self.output_layer(x)

    return x

model1=model()
print(model1)

model(
  (activation): ReLU()
  (conv_layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout2d(p=0, inplace=False)
  (conv_layer2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout2d(p=0, inplace=False)
  (conv_layer3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (batch_norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout2d(p=0, inplace=False)
  (conv_layer4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (batch_norm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout4): Dropout2d(p=0, inplace=False)
  (conv_layer5): Conv2d(256, 512, kernel_size=(7, 7), stride=(1, 1), p

In [24]:
epochs=100
learning_rate=0.1

In [25]:

loss=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model1.parameters(),lr=learning_rate)

model training function

In [34]:
def training(model1,data):
  loss_metric=nn.CrossEntropyLoss()
  optimizer=torch.optim.SGD(model1.parameters(),lr=learning_rate)

  model1.train()
  training_loss=0.0
  true_label=0
  total_train=0

  for input, label in data:


    input = input.to(device)
    model1 = model1.to(device)
    optimizer.zero_grad()

    output=model1(input)
    loss=loss_metric(output,label)
    loss.backward()
    optimizer.step()
    training_loss += loss.item()
    _,predicted=torch.max(output.data,1)
    total_train += label.size(0)
    true_label += (predicted==label).sum().item()

  train_accuracy=100*true_label/total_train
  return train_accuracy,training_loss,model1


model testing function(on validation data)

In [27]:
def test_on_valid_data(model, test_data):
    model.eval()

    correct_label = 0
    total_label = 0
    with torch.no_grad():
        for img, label in test_data:
            img, label = img.to(device), label.to(device)
            output = model(img)

            _, pred = torch.max(output, 1)
            correct_label += (pred == label).sum().item()
            total_label += label.size(0)

    valid_accuracy = 100 * correct_label / total_label
    return valid_accuracy


In [28]:
def model_train_val(model, train_data, val_data,epochs):

    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        train_accuracy,avg_loss,model1 = training(model, train_data)
        # Print training loss and accuracy
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

        # Validation loop
        val_accuracy = test_on_valid_data(model, val_data)
        # Print validation accuracy
        print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%')


        wandb.log({'Train loss': avg_loss})
        wandb.log({'Train accuracy': train_accuracy})

        wandb.log({'val_accuracy': val_accuracy})
        wandb.log({'epoch': epoch})


In [29]:
!pip install wandb

e4d0a8c3ccaf2534e9ab91c659e420ba5114533f

> Add blockquote



In [30]:
import wandb
wandb.login()

True

In [31]:


sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3],[3,5,5,7,7],[3,5,3,5,7],[5,5,5,5,5]]#,[7,7,7,7,7]]
        },
        'dropout': {
            'values': [0.3, 0.2]
        },
        'activation': {
            'values': [ 'relu','mish','silu', 'gelu',]
        },
        'num_dense':{
            'values': [128, 256]
        },
        'batch_norm':{
            'values': ['Yes','No']
        },
        'filter_org':{
            'values': [[128,128,64,64,32],[32,64,128,256,512],[32,32,32,32,32],[32,64,64,128,128]]
        },
        'learning_rate':{
            'values': [0.001,0.0001]
        },
        'optimizer':{
            'values': ['Adam','SGD']
        },
        'data_aug': {
            'values': ['No', 'Yes']
        }

    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='DL A2')

Create sweep with ID: al6ebpim
Sweep URL: https://wandb.ai/mfazal735-iit-madras-foundation/DL%20A2/sweeps/al6ebpim


In [35]:
def main():

    with wandb.init() as run:
        run_name="ks"+str(wandb.config.kernel_size)+"ac-"+(wandb.config.activation)+"_drop-"+str(wandb.config.dropout)+"_fs-"+str(wandb.config.filter_org)+"_bn-"+str(wandb.config.batch_norm)+"_dence-"+str(wandb.config.num_dense)
        wandb.run.name=run_name


        if  wandb.config.activation == 'relu':
            activ=nn.ReLU()
        elif wandb.config.activation == 'gelu':
            activ=nn.GELU()
        elif wandb.config.activation == 'silu':
            activ=nn.SiLU()
        elif wandb.config.activation == 'mish':
            activ=nn.Mish()

        model_= model(num_filters=wandb.config.filter_org, filter_size=wandb.config.kernel_size,
                      activation=activ, stride=1,padding=1, pool_size=(2,2), fc_size=wandb.config.num_dense,
                      nom_o_classes=10,dropout = wandb.config.dropout).to(device)

        train, validation = train_data(train_data_dir,data_augmentation= wandb.config.data_aug)

        model_train_val(model=model_, train_data=train, val_data=validation, epochs = 100)

wandb.agent(sweep_id, function= main,count= 10)
wandb.finish()

wandb: Agent Starting Run: doixdaf9 with config:
wandb: 	activation: silu
wandb: 	batch_norm: No
wandb: 	data_aug: No
wandb: 	dropout: 0.3
wandb: 	filter_org: [32, 64, 64, 128, 128]
wandb: 	kernel_size: [3, 3, 3, 3, 3]
wandb: 	learning_rate: 0.0001
wandb: 	num_dense: 256
wandb: 	optimizer: SGD


Traceback (most recent call last):
  File "<ipython-input-35-cc5c5e4323dd>", line 23, in main
    model_train_val(model=model_, train_data=train, val_data=validation, epochs = 100)
  File "<ipython-input-28-fbb3aa9b0fd8>", line 8, in model_train_val
    train_accuracy,avg_loss,model1 = training(model, train_data)
                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-34-2d0f18e92c85>", line 18, in training
    loss=loss_metric(output,label)
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py", line 1295, in forward
    retu

wandb: ERROR Run doixdaf9 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-35-cc5c5e4323dd>", line 23, in main
wandb: ERROR     model_train_val(model=model_, train_data=train, val_data=validation, epochs = 100)
wandb: ERROR   File "<ipython-input-28-fbb3aa9b0fd8>", line 8, in model_train_val
wandb: ERROR     train_accuracy,avg_loss,model1 = training(model, train_data)
wandb: ERROR                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "<ipython-input-34-2d0f18e92c85>", line 18, in training
wandb: ERROR     loss=loss_metric(output,label)
wandb: ERROR          ^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
wandb: ERROR     return self._call_impl(*args, **kwargs)
wandb: ERROR